In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pytickersymbols import PyTickerSymbols

pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def test(ticker):
    data = yf.download(ticker, period = "1y", interval = "1d")
    
    #Get daily closing prices
    daily_closing_prices = data['Close']
    
    # Calculate daily returns (percentage change)
    daily_returns = daily_closing_prices.pct_change().dropna()

    # Calculate annualized return (based on 252 trading days in a year)
    annualized_return = ((1 + daily_returns.mean()) ** 252) - 1
    
    # Calculate annualized standard deviation (volatility)
    annualized_volatility = daily_returns.std() * np.sqrt(252)
    
    # Sharpe ratio formula: (Expected Return - Risk-Free Rate) / Volatility
    risk_free_rate = 0.045
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    
    return sharpe_ratio




In [ ]:
nvda = (test("NVDA"))
appl = (test("AAPL"))
mean = (nvda + appl)/2

print(mean)

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def calculate_portfolio_sharpe_ratio(stock_symbols, risk_free_rate=0.045):
    """
    Calculate the Sharpe ratio of a portfolio over the past year.
    
    :param stock_symbols: List of stock ticker symbols (e.g., ['AAPL', 'NVDA'])
    :param weights: List of portfolio weights for each stock (e.g., [0.5, 0.5])
    :param risk_free_rate: Annual risk-free rate (default is 5%)
    :return: Sharpe ratio, annualized return, and annualized volatility of the portfolio
    """    
    # Fetch historical data for all stocks in the portfolio
    stock_data = yf.download(stock_symbols, period = "1y")['Close']

    if stock_data.isnull().values.any():
        print(f"Warning: Missing data for some stocks in the portfolio.")
        print(stock_data.isnull().sum())  # Print which stocks have missing data
        stock_data = stock_data.dropna(axis=1, how='any')  # Drop stocks with missing data


    # Calculate daily returns for each stock
    daily_returns = stock_data.pct_change().dropna()
    num = stock_data.shape[1]
    print(daily_returns)
    weight = [1 / num] * num

    weights = np.array(weight)

    # Calculate portfolio daily returns using the provided weights
    portfolio_daily_returns = daily_returns.dot(weights)

    # Calculate annualized return for the portfolio
    annualized_return = ((1 + portfolio_daily_returns.mean()) ** 252) - 1

    # Calculate annualized volatility (standard deviation) for the portfolio
    annualized_volatility = portfolio_daily_returns.std() * np.sqrt(252)
    
    # Sharpe ratio formula: (Expected Return - Risk-Free Rate) / Volatility
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility
    
    return sharpe_ratio, annualized_return, annualized_volatility

# Example usage:
#stock_symbols = ['AAPL', 'NVDA']  # Apple and Nvidia
#weights = [1, 1]  # Equal weighting
#risk_free_rate = float(input("Enter the risk-free rate (e.g., 0.05 for 5%): "))

#sharpe_ratio, annualized_return, annualized_volatility = calculate_portfolio_sharpe_ratio(stock_symbols, weights)

#print(f"\nFor portfolio of {stock_symbols}:")
#print(f"Sharpe Ratio: {sharpe_ratio:}")
#print(f"Annualized Return: {annualized_return:.2%}")
#print(f"Annualized Volatility: {annualized_volatility:.2%}")


In [ ]:
stock_data = PyTickerSymbols()
sp500_symbols = stock_data.get_stocks_by_index('S&P 500')
symbols = [stock['symbol'] for stock in sp500_symbols]
#symbols = ["TSLA", "NVDA", "AAPL", "MSFT", "MSTR", "AMZN", "META", "INTC"]

weight = [1 / 478] * 478
sharpe_ratio, annualized_return, annualized_volatility = calculate_portfolio_sharpe_ratio(symbols)

print(f"\nFor portfolio of {symbols}:")
print(f"Sharpe Ratio: {sharpe_ratio:}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Annualized Volatility: {annualized_volatility:.2%}")
